In [1]:
from pulp import *
import os
import numpy as np
import pandas as pd
import gurobipy as grb

def read_matrix(path_file):
    satisfactions = []
    resources = []
    capacities = []
    
    with open(path_file, 'r') as f:
        m, n= f.readline().split(' ')[:2]
        m, n = np.int32(m), np.int32(n)
        
        lines = f.readlines()
        lines = [line.rstrip().split(' ') for line in lines]

        for i in range(0, m):
            for j in range(n):
                satisfactions.append(np.int32(lines[i][j]))

        for i in range(m, m + m):
            for j in range(n):
                resources.append(np.int32(lines[i][j]))
        
        for j in range(m):
            capacities.append(np.int32(lines[2*m][j]))
            
    satisfactions = np.array(satisfactions)
    resources = np.array(resources)
    capacities = np.array(capacities)
    
    return m, n, satisfactions, resources, capacities

In [2]:
# References: 
# How to keep log files: https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html

In [2]:
import os
import json

def solve_fn(inputs):

    presolves = [0,1,2] # Controls the presolve level. A value of -1 corresponds to an automatic setting. Other options are off (0), conservative (1), or aggressive (2). More aggressive application of presolve takes more time, but can sometimes lead to a significantly tighter model.
    varbranches = [0,1,2,3] # Controls the branch variable selection strategy. The default -1 setting makes an automatic choice, depending on problem characteristics. Available alternatives are Pseudo Reduced Cost Branching (0), Pseudo Shadow Price Branching (1), Maximum Infeasibility Branching (2), and Strong Branching (3).Changing the value of this parameter rarely produces a significant benefit.
    all_cuts = [0,1,2,3] # Global cut aggressiveness setting. Use value 0 to shut off cuts, 1 for moderate cut generation, 2 for aggressive cut generation, and 3 for very aggressive cut generation. This parameter is overridden by the parameters that control individual cut types
    branchdirs = [0,1] # Determines which child node is explored first in the branch-and-cut search. The default value chooses automatically. A value of -1 will always explore the down branch first, while a value of 1 will always explore the up branch first.
    cover_cuts = [0,1,2]
    strong_cg_cuts = [0,1,2]
    clique_cuts = [0,1,2]

    results = []
    default = -1

    configs = [(default, default, default, default, i, default, default) for i in cover_cuts]
    configs += [(default, default, default, default, default, i, default) for i in strong_cg_cuts]
    configs += [(default, default, default, default, default, default, i) for i in clique_cuts]

    configs = [(i, default, default, default, default, default, default) for i in presolves] + [(default, i, default, default, default, default, default) for i in varbranches]
    configs += [(default, default, i, default, default, default, default) for i in all_cuts] + [(default, default, default, i, default, default, default) for i in branchdirs]
    configs += [(default, default, default, default, default, default, default)]
    
    for inputPath in inputs:
        for (presolve, varbranch, cuts, branchdir, cover_cut, strong_cg_cut, clique_cut) in configs:
            solver = getSolver('GUROBI', TimeLimit=180, Presolve=presolve, VarBranch=varbranch,
                                Cuts=cuts, BranchDir=branchdir, GUBCoverCuts=cover_cut, StrongCGCuts=strong_cg_cut,CliqueCuts=clique_cut)
            prob = LpProblem("desig_gen_problem", LpMaximize)

            workers, tasks, satisfaction, resource, capacity = read_matrix(os.path.join("./input", inputPath))

            N = range(workers*tasks)

            x = LpVariable.dicts(
                "x", N, lowBound=0, upBound=1, cat=LpInteger
            )

            prob += lpSum([satisfaction[i] * x[i]  for i in N]), 'objective F'

            #   every job can only be picked by one worker
            for i in range(tasks):
                prob += lpSum([x[i + j*tasks] for j in range(workers)]) == 1

            #   every worker has to take only the jobs that they can
            for i in range(workers):
                prob += lpSum([x[i*tasks + j] * resource[i*tasks + j] for j in range(tasks)]) <= capacity[i]
            
            prob.solve(solver)

            sol = json.loads(prob.solverModel.getJSONSolution())

            res = sol["SolutionInfo"]
            res['instance'] = inputPath
            res['presolve'] = presolve
            res['varbranch'] = varbranch
            res['cuts'] = cuts
            res['branchdir'] = branchdir

            res['cover_cut'] = cover_cut
            res['strong_cg_cut'] = strong_cg_cut
            res['clique_cut'] = clique_cut

            results.append(res)

    return results

In [4]:
results = solve_fn(os.listdir("./input/"))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-02
Set parameter TimeLimit to value 180
Set parameter BranchDir to value -1
Set parameter GUBCoverCuts to value 0
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1620 rows, 32000 columns and 64000 nonzeros
Model fingerprint: 0x439d68df
Variable types: 0 continuous, 32000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve time: 0.07s
Presolved: 1620 rows, 32000 columns, 64000 nonzeros
Variable types: 0 continuous, 32000 integer (32000 binary)
Found heuristic solution: objective -111777.0000

Root relaxation: objective -9.782135e+04, 4280 iterations, 0.12 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [ ]:
df_res = pd.DataFrame(results)
df_res.to_csv('res_generalized_assignment.csv', index=None)